<a href="https://colab.research.google.com/github/zboraon/anthropocene_date/blob/main/anthropocene_changepoint_simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Necessary libraries



Install and load the necessary libraries.


In [1]:
system("apt install -y jags")
system("apt install -y r-base")

In [ ]:
list.of.packages <- c("runjags", "mcp", "truncnorm", "coda", "dplyr", "ggplot2", "HDInterval")
new.packages <- list.of.packages[!(list.of.packages %in% installed.packages()[,"Package"])]
if(length(new.packages)) install.packages(new.packages, dependencies = TRUE)

In [3]:
rm(list = ls())


In [ ]:
library("mcp")
library("runjags")
library("dplyr")
library("HDInterval")  # For HDI calculation
library("coda")

set.seed(1952)


# Loading and tidying the data

In [5]:
model = list(
  y ~ 1,
  1 ~ 0 + x
)

empty = mcp(model, sample = FALSE)

df_sim = data.frame(
  x = runif(20, 1930, 1970),
  ageerror = abs(rnorm(20, mean = 0, sd = 1))
)

df_sim$y = empty$simulate(df_sim$x,
                      # Population-level:
                      int_1 =  0, x_2 = 0.25, cp_1 = 1952, sigma = 0.25)

y_values <- df_sim$y
df_sim$proxyerror = abs(rnorm(20, mean = 0, sd = 0.1))
yerror_values <- df_sim$proxyerror


fit_sim_to_plot <- data.frame(
  df_sim[, c("x", "ageerror")],
  y = y_values,
  y_err = yerror_values,
  id = rep(1,20)
)
colnames(fit_sim_to_plot) <- c("x", "x_err", "y", "y_err", "id")
fit_sim_to_plotsubset <- subset(fit_sim_to_plot, select = c("x", "x_err", "y", "y_err", "id"))


simulated_datatouse_selected_for_jags <- fit_sim_to_plotsubset %>%
  select(x, y, y_err, x_err, id) %>%
  rename(age = x, y = y, proxyerror = y_err, ageerror = x_err, id = id)

dataList_forjags <- as.list(simulated_datatouse_selected_for_jags)
dataList_forjags$minx <- min(simulated_datatouse_selected_for_jags$age)
dataList_forjags$maxx <- max(simulated_datatouse_selected_for_jags$age)

sampleNO <- 10000
thinstepsNO <- 2
adaptNO <- 5000
burninNO <- 5000
nChains <- 3

# The jags model

In [ ]:

modelString <- "
data {
  for (tr_ in 1:length(age)) {
  age_tr[tr_] <- age[tr_] / 1000
  ageerror_tr[tr_] <- ageerror[tr_] / 1000
  }
  MINX <- minx / 1000
  MAXX <- maxx / 1000
}
model {

  # Priors for population-level effects
  cp_0 = MINX  # mcp helper value.
  cp_2 = MAXX  # mcp helper value.

  cp_1_tr ~ dt(mu_cp_1, 1/sigma_cp_1^2, nu_cp)T(MINX, MAXX)
  sigma_cp_1 ~ dt(0, 0.05^(-2), 1)T(0, )
  int_1 ~ dt(0, 5^(-2), 3)
  x_2_tr ~ dt(0, 100^(-2), 3)
  sigma_1 ~  dt(0, 5^(-2), 1)T(0, )
  mu_cp_1 ~ dunif(MINX,MAXX)
  nu_cp ~ dexp(1/30)

  # Model and likelihood
  for (i_ in 1:length(age_tr)) {
    X_1_[i_] = min(trueage[i_], cp_1_tr)
    X_2_[i_] = min(trueage[i_], cp_2) - cp_1_tr
    trueage[i_] ~ dunif(MINX,MAXX) #dnorm(0, 1^(-2))
    age_tr[i_] ~  dnorm(trueage[i_], 1 / ageerror_tr[i_]^2)

    # Fitted value
    y_[i_] =

      # Segment 1: y ~ 1
      (trueage[i_] >= cp_0) * int_1 +

      # Segment 2: y ~ 1 ~ 0 + x
      (trueage[i_] >= cp_1_tr) * x_2_tr * X_2_[i_]

    # Fitted standard deviation
    tau_y_[i_] = (proxyerror[i_]^2 + (sigma_1)^2)^(-1)

    # Likelihood and log-density for family = gaussian()
    y[i_] ~ dnorm((y_[i_]), tau_y_[i_])
  }
  # Transform to original scale:
  cp_1 <- cp_1_tr * 1000
  # cp_1_sd <- cp_1_sigma_tr * 1000
  x_2 <- x_2_tr / 1000
}
"

writeLines(modelString, con="TEMPmodel.txt")
parameters <- c("cp_1", "int_1", "sigma_1", "x_2")


# run the model
runJagsOut_sim <- run.jags(method = "parallel",
                          model = "TEMPmodel.txt",
                          monitor = parameters,
                          data = dataList_forjags,
                          #inits = initsList,
                          n.chains = nChains,
                          adapt = adaptNO,
                          burnin = burninNO,
                          sample = sampleNO,
                          thin = thinstepsNO,
                          summarise = FALSE,
                          plots = FALSE,
                          modules = 'glm')

In [ ]:
      codaSamples <- as.mcmc.list(runJagsOut_sim)
      save(runJagsOut_sim, file="anthropocene_cp_sim_MCMC.Rdata" )
      summary2023colabrun_sim <- summary(runJagsOut_sim)
      write.csv(summary2023colabrun_sim, "/content/summary_sim_runs_sim.csv")